# Исследование объявлений о продаже квартир

В вашем распоряжении данные сервиса Яндекс Недвижимость — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктах за несколько лет. Вам нужно научиться определять рыночную стоимость объектов недвижимости. Для этого проведите исследовательский анализ данных и установите параметры, влияющие на цену объектов. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность.

По каждой квартире на продажу доступны два вида данных. Первые вписаны пользователем, вторые — получены автоматически на основе картографических данных. Например, расстояние до центра, аэропорта и других объектов — эти данные автоматически получены из геосервисов. Количество парков и водоёмов также заполняется без участия пользователя.

### Откройте файл с данными и изучите общую информацию

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
df_original = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
display(df)

In [ ]:
#изучим основную информацию
print(df.info())

In [ ]:
#Построим гистограммы
df.hist(figsize=(20, 20));

По гистограммам видно, что, например, в столбцах с площадями, высотой потолков, ценой есть большой расброс. Пропуски посчитаем ниже.

### Выполните предобработку данных

Посчитаем количество пропусков в каждом столбце:

In [ ]:
for column in df.columns:
    missing_data = df[column].isna().sum()
    print(f'Пропусков в столбце "{column}":{missing_data}')

Посчитаем количество дубликатов:

In [ ]:
df.duplicated().sum()

Начнем работу с пропусками:

Приведение времени к стандартному типу с точностью до дня:

In [ ]:
df['first_day_exposition'] = pd.to_datetime(df['first_day_exposition'])

Исключение строк с пропусками в столбце `floors_total`:

In [ ]:
df = df.dropna(subset=['floors_total']).reset_index(drop=True)

Исключение строк с пропусками в столбце `locality_name`:

In [ ]:
df = df.dropna(subset=['locality_name']).reset_index(drop=True)

Исключение объявлений с общей площадью меньше 20:

In [ ]:
df = df.drop(df.query('total_area < 20').index)

Для объявлений с высотой потолка больше 20 можно разделить эту высоту на 10. Остальные нереалистичные варианты исключим:

In [ ]:
df.loc[df['ceiling_height'] > 20,'ceiling_height'] = df['ceiling_height']/10

Хоромы с высотой потолка больше 7 и меньше 2 тоже надо исключить:

In [ ]:
df = df.drop(df.query('ceiling_height > 7').index)
df = df.drop(df.query('ceiling_height < 2').index)

Пропуски в балконах заполним нулями:

In [ ]:
df['balcony'] = df['balcony'].fillna(0)

Если в столбцах с ближайшими парками и водоемами пропуск, то, значит расстояних до них больше 3000 м. Заполним пропуски в этих столбцах 3000. Соответственно, в столбац с количеством парков и водоемов заменим пропуски нулями:

In [ ]:
df['parks_around3000'].fillna(0,inplace=True)
df['ponds_around3000'].fillna(0,inplace=True)

In [ ]:
missing_columns = ['parks_nearest','ponds_nearest']
for column in missing_columns:
    df[column] = df[column].fillna(3000)

Найдем и заменим неявные дубликаты в названиях н.п. Для удобства, заменим "ё" на "е". Приведем в порядок типы н.п. Заменим некоторые дублирующие названия.

In [ ]:
#Выведем количество уникальных топонимов
print(len(df['locality_name'].unique()))

#Заменим "ё" на "е"
df['locality_name'] = df['locality_name'].str.replace('ё', 'е')
#Заменим некоторые неявные дубликаты
df['locality_name'] = df['locality_name'].str.replace('поселок Мурино', 'Мурино')
df['locality_name'] = df['locality_name'].str.replace('поселок Рябово', 'поселок городского типа Рябово')

#Приведем к общему виду все типы населенных пунктов
df['locality_name'] = df['locality_name'].str.replace('городской поселок', 'поселок городского типа')
df['locality_name'] = df['locality_name'].str.replace('поселок станции', 'поселок при железнодорожной станции')
print(df['locality_name'].sort_values().unique())

#Выведем новое количество уникальных топонимов
print(len(df['locality_name'].sort_values().unique()))

Функция для создания столбца с типами н.п.:

In [ ]:
def locality_type_namer(locality):
    if 'поселок городского типа' in locality:
        return 'ПГТ'
    elif 'коттеджный поселок' in locality:
        return 'КП'
    elif 'деревня' in locality:
        return 'Село'
    elif 'поселок' in locality:
        return 'Село'
    elif 'поселок при железнодорожной станции' in locality:
        return 'Село'
    elif 'садовое' in locality:
        return 'СНТ'
    elif 'село' in locality:
        return 'Село'
    elif 'Санкт-Петербург' in locality:
        return 'Санкт-Петербург'
    else:
        return 'Город'

df['locality_type'] = df['locality_name'].apply(locality_type_namer)

print(df.tail(5))

Исключение объявленией, где сумма жилой площади и площади кухни больше общей площади:

In [ ]:
df = df.drop(df.query('living_area+kitchen_area > total_area').index)

Смотрим, что считается апартаментами:

In [ ]:
df[['total_area','rooms', 'is_apartment','locality_type']].query('is_apartment==True').sort_values('total_area')

Заполняем пропуски в `is_apartment`. Если комнат не ноль, то это апартаменты и наоборот:

In [ ]:
df.loc[df['rooms']>0,'is_apartment'] = True

In [ ]:
df.loc[df['rooms']==0,'is_apartment'] = False

In [ ]:
df.loc[df['rooms'] == 0, 'studio'] = True

In [ ]:
df.loc[df['rooms'] > 0, 'studio'] = False

In [ ]:
#Проверим на ошибки в логике:
df.loc[(df['is_apartment']==False)&(df['studio']==False)]

Заменим пропуски в `ceiling_height` медианой:

In [ ]:
print(df['ceiling_height'].median())
df['ceiling_height'].fillna(df['ceiling_height'].median(),inplace=True)

Проверим, нет ли ошибок в заполнении этажей

In [ ]:
df.query('floor > floors_total')

Начнем заполнять пропуски в площадях. Составим таблицу по комнатам и типу н.п., на основании которой заполним пропуски медианными значениям каждой группы:

In [ ]:
df.groupby(['rooms', 'locality_type']).mean()

In [ ]:
df.loc[df['living_area'].isna() & df['kitchen_area'].notna(), 'living_area'] = (df['total_area'] - df['kitchen_area'])
df.loc[df['kitchen_area'].isna() & df['living_area'].notna(), 'kitchen_area'] = (df['total_area'] - df['living_area'])

In [ ]:
df.loc[df['kitchen_area'].isna()]

Заполним столбцы `living_area` и `living_area` медианными значениями из соответствующих групп:

In [ ]:
#К каждому из требуемых столбцов применим метод transform('median') для сохранения исходных размеров таблицы: 
df['living_area'] = df['living_area'].fillna(df.groupby(['locality_type', 'rooms'])['living_area'].transform('median'))
df['kitchen_area'] = df['kitchen_area'].fillna(df.groupby(['locality_type', 'rooms'])['kitchen_area'].transform('median'))

В одной группе попалось одинокое объявление, которое нечем заполнить:

In [ ]:
df.loc[df['kitchen_area'].isna()]

Вручную посмотрим аналогичные параметры в предыдущей группе(`locality_type`=*село* `rooms`=*6*). Вычтем эти величины из известной общей площади и вставим в строку:

In [ ]:
df['kitchen_area'].fillna(17.8 ,inplace=True)
df['living_area'].fillna(183.2 ,inplace=True)

Заполним пропущенные расстояния до центра и до аэропорта медианными значениями группы по н.п.:

In [ ]:
df['airports_nearest'] = df['airports_nearest'].fillna(df.groupby(['locality_name'])['airports_nearest'].transform('median'))

In [ ]:
df['cityCenters_nearest'] = df['cityCenters_nearest'].fillna(df.groupby(['locality_name'])
                                                             ['cityCenters_nearest'].transform('median')
                                                            )

Остались незаполненные ячейки:

In [ ]:
df.loc[df['cityCenters_nearest'].isna()].groupby('locality_type').mean()

In [ ]:
df.loc[df['airports_nearest'].isna()].groupby('locality_type').mean()

Заполним их "значительным расстоянием":

In [ ]:
df['cityCenters_nearest'].fillna(65000,inplace=True)
df['airports_nearest'].fillna(800000,inplace=True)

Еще раз проверим наличие пропусков. Пропуски остались только в колонке `days_exposition`.

In [ ]:
for column in df.columns:
    missing_data = df[column].isna().sum()
    print(f'Пропусков в столбце "{column}":{missing_data}')

In [ ]:
df.hist(figsize=(20, 20));

In [ ]:
df.head(10)

### Добавьте в таблицу новые столбцы

Добавим столбец `price_for_square_meter` - средняя цена за квадратный метр:

In [ ]:
df['price_for_square_meter']=df['last_price']/df['total_area']

Теперь можно округлить и привести к целым некоторые столбцы:

In [ ]:
int_columns = ['last_price', 'floors_total', 'airports_nearest', 'cityCenters_nearest','parks_around3000', 'parks_nearest',
              'ponds_around3000', 'ponds_nearest','price_for_square_meter']
df[int_columns] = df[int_columns].astype(int)

Добавим столбцы с номером дня недели, месяца и года:

In [ ]:
df['weekday'] = df['first_day_exposition'].dt.weekday

In [ ]:
df['month'] = df['first_day_exposition'].dt.month

In [ ]:
df['year'] = df['first_day_exposition'].dt.year

Напишем функцию для добавления столбца `floor_type` - тип этажа:

In [ ]:
def floor_type_namer(row):
    if row['floor'] == 1:
        return 'первый этаж'
    elif row['floor'] == row['floors_total']:
        return 'последний этаж'
    else:
        return 'другой'

df['floor_type'] = df.apply(floor_type_namer,axis=1)


Переведем расстояния до центра в км, округлим и приведем к целочисленному значению:

In [ ]:
df['cityCenters_nearest'] = (df['cityCenters_nearest']/1000).round(0).astype(int)


### Проведите исследовательский анализ данных

Начнем выводить отдельные гистограммы по требуемым пунктам:

In [ ]:
df['total_area'].hist(bins=50);

Самая большая категория объявлений с общей площадью около 50 кв.м.

In [ ]:
df['living_area'].hist(bins=50);

In [ ]:
df['living_area'].hist(range=(0,100),bins=60);

Два пика: около 20 и 30 кв.м. Говорит о преобладании типовой застройки.

In [ ]:
df['kitchen_area'].hist(range=(0,40),bins=50);

Аналогично предыдущему: пики около 10 и 5 кв.м.

In [ ]:
df['last_price'].hist(bins=50);

In [ ]:
df['last_price'].hist(range=(0,5e7),bins=50);

В ценах разброс на два порядка. В основном квартиры стоят до 10 млн.

In [ ]:
df['rooms'].hist(range=(0,10),bins=50);

В основном в объявлениях одна-две комнаты.

In [ ]:
df['floor_type'].hist(bins=50);

Очевидно, что в основном квартиры продаются не на первом и последнем этаже, а где-то в промежутке.

In [ ]:
df['floors_total'].hist(bins=30);

Пики на 5 и 10 этажах говорят о наличии типовой застройки.

In [ ]:
df['ceiling_height'].hist(bins=50);

Большинство потолков такие, какие должны быть.

In [ ]:
df['cityCenters_nearest'].hist(bins=50);

В основном квартиры очень далеко от центра. 

In [ ]:
df['parks_nearest'].hist(bins=50);

Парков в радиусе 3 км мало.

In [ ]:
df['parks_nearest'].hist(range=(0,2999),bins=50);

Если парк есть, то он скорее всего на расстоянии 500 м. 

Разберемся со столбцом `last_price`:

In [ ]:
df['last_price'].hist(bins=50);

In [ ]:
df['last_price'].hist(range=(6e7,8e8),bins=50);

In [ ]:
df[['last_price', 'cityCenters_nearest','total_area','locality_type']].sort_values(['last_price'],ascending=False)

В графике выше аномалиями я бы назвал первое и последнее объявление. Первое слишком дорого в сравнении со следующими объявлениями, хотя тут опять же надо смотреть на данные, которых в таблице нет. Последнее слишком дешевое, даже с учетом, что там цена может быть ошибочно записана в тысячах рублей. Удалим эти столбцы:

In [ ]:
df = df.drop([12897, 8750]).reset_index(drop=True)

In [ ]:
df['last_price'].hist(bins=50);

У гистограммы есть хвост после 100 млн руб, удалим эти значения:

In [ ]:
df.query('last_price > 1e8')

In [ ]:
#Исправил согласно v2
df = df.drop(df.query('last_price > 1e8').index).reset_index(drop=True)

In [ ]:
df['last_price'].hist(bins=50);

Разберемся со столбцом `rooms`:

In [ ]:
df[['last_price', 'cityCenters_nearest','total_area','locality_type','rooms']].sort_values(['rooms'],ascending=False)

Есть перебор с комнатами. В задании указано, что продаются квартиры, так что помещения с таким большим количеством комнат можно считать коммерческими. По гистограмме комнат видно, что есть маленькая группа объявлений, где больше 8 комнат:

In [ ]:
df.query('rooms > 8')

Удалим их:

In [ ]:
#Исправил согласно v2
df = df.drop(df.query('rooms > 8').index).reset_index(drop=True)

In [ ]:
df[['last_price', 'cityCenters_nearest','total_area','locality_type','rooms','price_for_square_meter']].sort_values(['total_area'],ascending=False)

Квартиры больше 250 кв.м выглядит странновато, можно тоже удалить

In [ ]:
#Исправил согласно v2
df = df.drop(df.query('total_area > 250').index).reset_index(drop=True)

Оценим процент удаленных данных:

In [ ]:
print(f"Процент удаленных данных: {(1 - (df.shape[0] / df_original.shape[0]))*100:.2f}%")

Выглядит терпимо.

In [ ]:
#код ревьюера
data2 = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
(data2['total_area']).hist(bins=30);

Изучим, как быстро продавались квартиры:

In [ ]:
df['days_exposition'].hist(bins=50);

In [ ]:
print(df['days_exposition'].describe()) 

В среднем квартира продается 180 дней. По медиане 95 дней. Категории продаж можно разделить по квартилям:
Быстрые продажи: до первого квартиля (до 45 дней);
Обычные продажи: между первым и третьим квартилем (от 45 до 232 дней);
Медленные продажи: после третьего квартиля (от 232 дней).

In [ ]:
df['days_exposition'].hist(range=(0,100),bins=50);

Выведем графики зависимости цены от требуемых параметров. Сразу будем считать корреляцию между этими параметрами:

In [ ]:
df.plot(x='total_area', y='last_price', kind='scatter', alpha=0.7);

In [ ]:
df['total_area'].corr(df['last_price'])

In [ ]:
df.plot(x='living_area', y='last_price', kind='scatter', alpha=0.7);

In [ ]:
df['living_area'].corr(df['last_price'])

In [ ]:
df.plot(x='kitchen_area', y='last_price', kind='scatter', alpha=0.7);

In [ ]:
df['kitchen_area'].corr(df['last_price'])

In [ ]:
df.plot(x='rooms', y='last_price',  kind='scatter', alpha=0.7);

In [ ]:
df['rooms'].corr(df['last_price'])

In [ ]:
corr_df=df[['last_price', 'total_area', 'kitchen_area', 'living_area', 'rooms']]
print(corr_df.corr()['last_price'])

In [ ]:
#Средняя цена по типу этажа
mean_price_floor_type = df.groupby('floor_type')['last_price'].mean()

#Построение графика средней цены по этажам
mean_price_floor_type.plot(kind='bar', title='Средняя цена по типу этажа', ylabel='Средняя цена', xlabel='Тип этажа')
plt.show()

#Средняя цена по дням
mean_price_weekday = df.groupby('weekday')['last_price'].mean()

#Построение графика по дням
mean_price_weekday.plot(kind='bar', title='Средняя цена по месяцам', ylabel='Средняя цена', xlabel='Месяц')
plt.show()

#Средняя цена по месяцам
mean_price_month = df.groupby('month')['last_price'].mean()

#Построение графика по месяцам
mean_price_month.plot(kind='bar', title='Средняя цена по месяцам', ylabel='Средняя цена', xlabel='Месяц')
plt.show()

#Средняя цена по годам
mean_price_year = df.groupby('year')['last_price'].mean()

#Построение графика по годам
mean_price_year.plot(kind='bar', title='Средняя цена по годам', ylabel='Средняя цена', xlabel='Год')
plt.show()

Построим то же самое по медиане:

In [ ]:
#Медианая цена по типу этажа
median_price_floor_type = df.groupby('floor_type')['last_price'].median()

#Построение графика медианной цены по этажам
median_price_floor_type.plot(kind='bar', title='Медианная цена по типу этажа', ylabel='Медианная цена', xlabel='Тип этажа')
plt.show()

#Медианая цена по дням
median_price_weekday = df.groupby('weekday')['last_price'].median()

#Построение графика по дням
median_price_weekday.plot(kind='bar', title='Медианная цена по дням', ylabel='Медианная цена', xlabel='День')
plt.show()

#Медианая цена по месяцам
median_price_month = df.groupby('month')['last_price'].median()

#Построение графика по месяцам
median_price_month.plot(kind='bar', title='Медианная цена по месяцам', ylabel='Медианная цена', xlabel='Месяц')
plt.show()

#Медианая цена по годам
median_price_year = df.groupby('year')['last_price'].median()

#Построение графика по годам
median_price_year.plot(kind='bar', title='Медианная цена по годам', ylabel='Медианная цена', xlabel='Год')
plt.show()

In [ ]:
#Количество объявлений в год
count_year = df.groupby('year')['last_price'].count()

#Построение графика по годам
count_year.plot(kind='bar', title='Количество объявлений в год', ylabel='Количество объявлений', xlabel='Год')
plt.show()

График это подтверждает. В 2014 году количество объявлений исчислялось сотнями, а в 2015 тысячами.

Сильнее всего стоимость зависит от общей площади (0.65), однако это суммарный параметр, который состоит из жилой площади (0.57) и площади кухни (0.49). Жилая площадь на стоимость квартиры влияет чуть сильнее, чем площадь кухни. Количество комнат слабо влияет на стоимость квартиры (0.36).

Посчитаем среднюю цену одного квадратного метра в 10 населённых пунктах с наибольшим числом объявлений.

In [ ]:
#group_by_price = df.groupby('locality_name').agg({
#    'price_for_square_meter': 'mean',  # Средняя цена за квадратный метр
#    'locality_name': 'count'  # Количество объявлений
#}).rename(columns={'locality_name': 'locality_count'})

#print(group_by_price.sort_values('locality_count', ascending=False).head(10))

low_price = df.pivot_table(index='locality_name', values='price_for_square_meter', 
                           aggfunc=({'price_for_square_meter': 'mean',
                                'locality_name': 'count'})).rename(columns={'locality_name': 'locality_count'})
low_price['price_for_square_meter']=low_price['price_for_square_meter'].round(-1).astype(int)
low_price =  low_price.sort_values('locality_count', ascending=False).head(10)
print(low_price.sort_values('price_for_square_meter', ascending=False).head(10))

Больше всего объявлений (15577 шт.) в Санкт-Петербурге, там же и самый дорогой квадратный метр (114670 руб/кв.м). Замыкает десятку Выборг (237 шт.) (58140 руб/кв.м)

Построим график зависимости цены за кв.м. от расстояния до центра СПБ

In [ ]:
#С помощью логической индексации выберем в нашу таблицу объявления из СПБ
price_per_km_spb = df[df['locality_name'] == 'Санкт-Петербург']

#Сгруппируем таблицу по `cityCenters_nearest` и посчитаем среднее. Восстановим индексацию
price_per_km_spb = price_per_km_spb.groupby('cityCenters_nearest').agg({'last_price': 'mean'}).reset_index()

# Переименование колонок для удобства
price_per_km_spb.columns = ['cityCenters_distance', 'price']

# Построение графика
price_per_km_spb.plot(x='cityCenters_distance', y='price', style='o-',grid=True, 
        title='Зависимость средней цены квартир от расстояния до центра Санкт-Петербурга',
        xlabel='Расстояние до центра, км',
        ylabel='Средняя цена, руб'
       )


В центре квартиры более дорогие, на удалении от центра дешевеют до определенного значения с учетом окружающей местности. Всплески показывают наличие всяких элитных жк, новых районов и прочего.

Выведем все коэффициенты корреляции с ценой:

In [ ]:
df_corr = df.corr()['last_price'].sort_values(ascending=False)

print(df_corr)

Выведем все коэффициенты корреляции с ценой для Санкт-Петербурга:

In [ ]:
spb_corr=df.loc[df['locality_name']=='Санкт-Петербург'].corr()['last_price'].sort_values(ascending=False)

print(spb_corr)

Сделаем единую таблицу корреляций:

In [ ]:
spb_vs_all=pd.concat({'Корреляция по всей таблице': df_corr, 'Корреляция по СПБ': spb_corr}, axis=1)
spb_vs_all.sort_values(['Корреляция по всей таблице'],ascending=False)

### Вывод

В результате обработки данных было отброшено 1.56% данных.

Сильнее всего на стоимость квартиры влияет ее площадь(коэффициент корреляции = 0.65), при этом жилая площадь(0.57) важнее площади кухни (0.49). Среднее влияние оказывает количество комнат (0.36), высота потолков (0.29), удаленность от центра (-0.24). Несущественное влияние оказывает наличие парков и водоемов и удаленность от них. Так же на цену сильно влияет тип этажа, что не отражает корреляция. Так же в Санкт-Петербурге менее важно(-0.18 по всей таблице против -0.014 в СПБ) расстояние до аэропорта, видимо, из-за развитой транспортной системы. Наличие и расстояние до парков и водоемов тоже не так важно для ценообразования в СПБ. 
На графике со среднем значением цены по типу этажа наблюдаются завышения в категории последний этаж. Скорее всего это говорит о наличии объявлений с очень дорогими квартирами, занимающими последний или последние этажи. В целом медианные графики "ниже".

В 2014 году квартиры в среднем и по медиане были очень дорогими, а потом сильно подешевели. Скорее всего сервис заработал в 2014 году и там почти ничего не было, а к 2015 году появилось много дешевых объявлений из области и в принципе по городу. К 2019 году квартиры несколько подорожали.

Расстояние до центра в Санкт-Петербурге быстро падает с 30 млн до 5-6 млн и далее почти не меняется. Небольшие скачки говорят о более дорогих районах, жк, близости к достопримечательностям, которые не учтены в этой таблице.

В среднем квартира продается 180 дней. По медиане 95 дней. Категории продаж можно разделить по квартилям: Быстрые продажи: до первого квартиля (до 45 дней); Обычные продажи: между первым и третьим квартилем (от 45 до 232 дней); Медленные продажи: после третьего квартиля (от 232 дней).

Больше всего объявлений (15577 шт.) в Санкт-Петербурге, там же и самый дорогой квадратный метр (114670 руб/кв.м). Замыкает десятку Выборг (237 шт.) (58140 руб/кв.м).

Для улучшения анализа можно, например, сделать скейлинг(полином) от всех параметров и рассчитывать примерную рыночную цену квартиры.